# Word2Vec Tutorial

In case you missed the buzz, word2vec is a widely featured as a member of the “new wave” of machine learning algorithms based on neural networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow). Using large amounts of unannotated plain text, word2vec learns relationships between words automatically. The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”), or vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.

Word2vec is very useful in [automatic text tagging](https://github.com/RaRe-Technologies/movie-plots-by-genre), recommender systems and machine translation.

Check out an [online word2vec demo](http://radimrehurek.com/2014/02/word2vec-tutorial/#app) where you can try this vector algebra for yourself. That demo runs `word2vec` on the Google News dataset, of **about 100 billion words**.

## This tutorial

In this tutorial you will learn how to train and evaluate word2vec models on your business data.


## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings):

In [2]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-06-22 10:43:52,078 : INFO : collecting all words and their counts
2017-06-22 10:43:52,082 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-22 10:43:52,084 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2017-06-22 10:43:52,086 : INFO : Loading a fresh vocabulary
2017-06-22 10:43:52,088 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-06-22 10:43:52,090 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-06-22 10:43:52,092 : INFO : deleting the raw counts dictionary of 3 items
2017-06-22 10:43:52,095 : INFO : sample=0.001 downsamples 3 most-common words
2017-06-22 10:43:52,097 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-06-22 10:43:52,099 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-06-22 10:43:52,100 : INFO : resetting layer weights
2017-06-22 10:43:52,120 : INFO : training model with 3 workers o

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, when iterated over. No need to keep everything in RAM: we can provide one sentence, process it, forget it, load another sentence…

For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [4]:
# create some toy data to use with the following example
import smart_open, os

if not os.path.exists('./data/'):
    os.makedirs('./data/')

filenames = ['./data/f1.txt', './data/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [5]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

In [6]:
sentences = MySentences('./data/') # a memory-friendly iterator
print(list(sentences))

[['first'], ['sentence'], ['second'], ['sentence']]


In [7]:
# generate the Word2Vec model
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-06-22 10:43:56,495 : INFO : collecting all words and their counts
2017-06-22 10:43:56,500 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-22 10:43:56,506 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-06-22 10:43:56,507 : INFO : Loading a fresh vocabulary
2017-06-22 10:43:56,508 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-06-22 10:43:56,509 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-06-22 10:43:56,511 : INFO : deleting the raw counts dictionary of 3 items
2017-06-22 10:43:56,512 : INFO : sample=0.001 downsamples 3 most-common words
2017-06-22 10:43:56,513 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-06-22 10:43:56,514 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-06-22 10:43:56,515 : INFO : resetting layer weights
2017-06-22 10:43:56,517 : INFO : training model with 3 workers o

In [8]:
print(model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'first': <gensim.models.keyedvectors.Vocab object at 0x7ff773e30978>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x7ff773e30ac8>, 'second': <gensim.models.keyedvectors.Vocab object at 0x7ff773dcbf60>}


Say we want to further preprocess the words from the files — convert to unicode, lowercase, remove numbers, extract named entities… All of this can be done inside the `MySentences` iterator and `word2vec` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.

**Note to advanced users:** calling `Word2Vec(sentences, iter=1)` will run **two** passes over the sentences iterator. In general it runs `iter+1` passes. By the way, the default value is `iter=5` to comply with Google's word2vec in C language. 
  1. The first pass collects words and their frequencies to build an internal dictionary tree structure. 
  2. The second pass trains the neural model.

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [9]:
# build the same model, making the 2 steps explicit
new_model = gensim.models.Word2Vec(min_count=1)  # an empty model, no training
new_model.build_vocab(sentences)                 # can be a non-repeatable, 1-pass generator     
new_model.train(sentences, total_examples=new_model.corpus_count, epochs=new_model.iter)                       
# can be a non-repeatable, 1-pass generator

2017-06-22 10:43:59,156 : INFO : collecting all words and their counts
2017-06-22 10:43:59,158 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-22 10:43:59,160 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-06-22 10:43:59,161 : INFO : Loading a fresh vocabulary
2017-06-22 10:43:59,162 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-06-22 10:43:59,162 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-06-22 10:43:59,163 : INFO : deleting the raw counts dictionary of 3 items
2017-06-22 10:43:59,164 : INFO : sample=0.001 downsamples 3 most-common words
2017-06-22 10:43:59,165 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-06-22 10:43:59,166 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-06-22 10:43:59,167 : INFO : resetting layer weights
2017-06-22 10:43:59,169 : INFO : training model with 3 workers o

0

In [10]:
print(new_model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'first': <gensim.models.keyedvectors.Vocab object at 0x7ff773e30978>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x7ff773e30ac8>, 'second': <gensim.models.keyedvectors.Vocab object at 0x7ff773dcbf60>}


### More data would be nice
For the following examples, we'll use the [Lee Corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor) (which you already have if you've installed gensim):

In [11]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [12]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

### min_count
`min_count` is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [13]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

2017-06-22 10:44:03,482 : INFO : collecting all words and their counts
2017-06-22 10:44:03,594 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-22 10:44:03,654 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-06-22 10:44:03,655 : INFO : Loading a fresh vocabulary
2017-06-22 10:44:03,667 : INFO : min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2017-06-22 10:44:03,669 : INFO : min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2017-06-22 10:44:03,675 : INFO : deleting the raw counts dictionary of 10186 items
2017-06-22 10:44:03,680 : INFO : sample=0.001 downsamples 54 most-common words
2017-06-22 10:44:03,681 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2017-06-22 10:44:03,683 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2017-06-22 10:44:03,689 : INFO : resetting layer weights
2017-06-22 10:4

### size
`size` is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

In [14]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

2017-06-22 10:44:05,731 : INFO : collecting all words and their counts
2017-06-22 10:44:05,733 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-22 10:44:05,762 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-06-22 10:44:05,763 : INFO : Loading a fresh vocabulary
2017-06-22 10:44:05,779 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-06-22 10:44:05,780 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-06-22 10:44:05,792 : INFO : deleting the raw counts dictionary of 10186 items
2017-06-22 10:44:05,795 : INFO : sample=0.001 downsamples 49 most-common words
2017-06-22 10:44:05,795 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-06-22 10:44:05,798 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2017-06-22 10:44:05,807 : INFO : resetting layer weights
2017-06-22 10:

### workers
`workers`, the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [15]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2017-06-22 10:44:07,835 : INFO : collecting all words and their counts
2017-06-22 10:44:07,839 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-22 10:44:07,876 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-06-22 10:44:07,879 : INFO : Loading a fresh vocabulary
2017-06-22 10:44:07,898 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-06-22 10:44:07,900 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-06-22 10:44:07,914 : INFO : deleting the raw counts dictionary of 10186 items
2017-06-22 10:44:07,917 : INFO : sample=0.001 downsamples 49 most-common words
2017-06-22 10:44:07,918 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-06-22 10:44:07,920 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2017-06-22 10:44:07,932 : INFO : resetting layer weights
2017-06-22 10:

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

Gensim supports the same evaluation set, in exactly the same format:

In [16]:
model.accuracy('./datasets/questions-words.txt')

2017-06-22 10:44:11,035 : INFO : precomputing L2-norms of word weight vectors
2017-06-22 10:44:11,247 : INFO : family: 0.0% (0/2)
2017-06-22 10:44:11,289 : INFO : gram3-comparative: 0.0% (0/12)
2017-06-22 10:44:11,310 : INFO : gram4-superlative: 0.0% (0/12)
2017-06-22 10:44:11,331 : INFO : gram5-present-participle: 0.0% (0/20)
2017-06-22 10:44:11,361 : INFO : gram6-nationality-adjective: 0.0% (0/20)
2017-06-22 10:44:11,389 : INFO : gram7-past-tense: 0.0% (0/20)
2017-06-22 10:44:11,410 : INFO : gram8-plural: 0.0% (0/12)
2017-06-22 10:44:11,422 : INFO : total: 0.0% (0/98)


[{'correct': [], 'incorrect': [], 'section': 'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': 'capital-world'},
 {'correct': [], 'incorrect': [], 'section': 'currency'},
 {'correct': [], 'incorrect': [], 'section': 'city-in-state'},
 {'correct': [],
  'incorrect': [('HE', 'SHE', 'HIS', 'HER'), ('HIS', 'HER', 'HE', 'SHE')],
  'section': 'family'},
 {'correct': [], 'incorrect': [], 'section': 'gram1-adjective-to-adverb'},
 {'correct': [], 'incorrect': [], 'section': 'gram2-opposite'},
 {'correct': [],
  'incorrect': [('GOOD', 'BETTER', 'GREAT', 'GREATER'),
   ('GOOD', 'BETTER', 'LONG', 'LONGER'),
   ('GOOD', 'BETTER', 'LOW', 'LOWER'),
   ('GREAT', 'GREATER', 'LONG', 'LONGER'),
   ('GREAT', 'GREATER', 'LOW', 'LOWER'),
   ('GREAT', 'GREATER', 'GOOD', 'BETTER'),
   ('LONG', 'LONGER', 'LOW', 'LOWER'),
   ('LONG', 'LONGER', 'GOOD', 'BETTER'),
   ('LONG', 'LONGER', 'GREAT', 'GREATER'),
   ('LOW', 'LOWER', 'GOOD', 'BETTER'),
   ('LOW', 'LOWER', 'GREAT', 'GREATER'),
   (

This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset specific to your business based on it. It contains word pairs together with human-assigned similarity judgments. It measures the relatedness or co-occurrence of two words. For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [17]:
model.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

2017-06-22 10:44:13,512 : INFO : Pearson correlation coefficient against /home/aneeshjoshi/Anaconda3.5install/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv: 0.0979
2017-06-22 10:44:13,513 : INFO : Spearman rank-order correlation coefficient against /home/aneeshjoshi/Anaconda3.5install/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv: 0.1338
2017-06-22 10:44:13,514 : INFO : Pairs with unknown words ratio: 85.6%


((0.097870577722243585, 0.49444801252988957),
 SpearmanrResult(correlation=0.13381604284704934, pvalue=0.34918777369466802),
 85.55240793201133)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [18]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

2017-06-22 10:44:16,764 : INFO : saving Word2Vec object under /tmp/tmp05ra0oc9gensim_temp, separately None
2017-06-22 10:44:16,768 : INFO : not storing attribute syn0norm
2017-06-22 10:44:16,771 : INFO : not storing attribute cum_table
2017-06-22 10:44:16,805 : INFO : saved /tmp/tmp05ra0oc9gensim_temp


In [19]:
new_model = gensim.models.Word2Vec.load(temp_path)  # open the model

2017-06-22 10:44:17,437 : INFO : loading Word2Vec object from /tmp/tmp05ra0oc9gensim_temp
2017-06-22 10:44:17,539 : INFO : loading wv recursively from /tmp/tmp05ra0oc9gensim_temp.wv.* with mmap=None
2017-06-22 10:44:17,540 : INFO : setting ignored attribute syn0norm to None
2017-06-22 10:44:17,541 : INFO : setting ignored attribute cum_table to None
2017-06-22 10:44:17,542 : INFO : loaded /tmp/tmp05ra0oc9gensim_temp


which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:
```
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip:
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences and [new vocabulary words](online_w2v_tutorial.ipynb):

In [20]:
model = gensim.models.Word2Vec.load(temp_path)
more_sentences = [['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.iter)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2017-06-22 10:44:22,860 : INFO : loading Word2Vec object from /tmp/tmp05ra0oc9gensim_temp
2017-06-22 10:44:22,890 : INFO : loading wv recursively from /tmp/tmp05ra0oc9gensim_temp.wv.* with mmap=None
2017-06-22 10:44:22,891 : INFO : setting ignored attribute syn0norm to None
2017-06-22 10:44:22,896 : INFO : setting ignored attribute cum_table to None
2017-06-22 10:44:22,898 : INFO : loaded /tmp/tmp05ra0oc9gensim_temp
2017-06-22 10:44:22,912 : INFO : collecting all words and their counts
2017-06-22 10:44:22,913 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-22 10:44:22,914 : INFO : collected 13 word types from a corpus of 13 raw words and 1 sentences
2017-06-22 10:44:22,916 : INFO : Updating model with new vocabulary
2017-06-22 10:44:22,917 : INFO : New added 0 unique words (0% of original 13)
                        and increased the count of 0 pre-existing words (0% of original 13)
2017-06-22 10:44:22,918 : INFO : deleting the raw counts dictionary o

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [23]:
model.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

[('information', 0.9931168556213379)]

In [24]:
model.doesnt_match("input is lunch he sentence cat".split())

2017-06-22 10:44:39,560 : WARNING : vectors for words {'input', 'cat', 'lunch'} are not present in the model, ignoring these words


'sentence'

In [25]:
print(model.similarity('human', 'party'))
print(model.similarity('tree', 'murder'))

0.998855094888
0.996164904471


You can get the probability distribution for the center word given the context words as input:

In [26]:
print(model.predict_output_word(['emergency', 'beacon', 'received']))

[('more', 0.0010606901), ('training', 0.00094614469), ('continue', 0.0009013316), ('can', 0.0008800436), ('it', 0.00077920244), ('she', 0.00077671453), ('australia', 0.00077293179), ('arafat', 0.00075185142), ('could', 0.00075145147), ('when', 0.00074343773)]


The results here don't look good because the training corpus is very small. To get meaningful results one needs to train on 500k+ words.

If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [27]:
model['tree']  # raw NumPy vector of a word

array([-0.07387707, -0.0525472 , -0.00801231, -0.02258717,  0.02612364,
       -0.04270453,  0.02020921, -0.05816618,  0.05283009, -0.03610224,
       -0.04228791, -0.06469296, -0.01243895, -0.00832417, -0.03136453,
        0.02577102,  0.04021412, -0.01404949, -0.03235153, -0.02063898,
        0.00729888,  0.04992125,  0.03474784, -0.05255898, -0.05749521,
       -0.02488662, -0.022747  ,  0.00581837,  0.02808935,  0.07712489,
        0.02640385,  0.0214168 ,  0.01918079,  0.00955962,  0.02823325,
        0.0569525 , -0.03393821,  0.039653  ,  0.0271197 , -0.02554145,
       -0.01306454,  0.00434697, -0.00493329,  0.02808762,  0.01226015,
        0.01681786,  0.05796145,  0.00522719,  0.01701758,  0.02785488,
       -0.0016802 , -0.05068791,  0.00986914,  0.06035094,  0.00126926,
       -0.00550077,  0.01481648, -0.02769886,  0.04022276, -0.12319525,
        0.01072143,  0.1151017 ,  0.01084812, -0.03169062, -0.03724645,
        0.00609815, -0.06609762, -0.03607272, -0.03780094,  0.00

…or en-masse as a 2D NumPy matrix from `model.wv.syn0`.

# Visualising the Word Embeddings

The word embeddings made by the model can be visualised by reducing dimensionality of the words to 2 dimensions using tSNE.
These 2 dimensional word vectors can then be plotted and give a good idea about the layout of the words

Additional dependencies : 
- sklearn
- numpy
- plotly

The function below can be used to plot the embeddings in an ipython notebook.
It requires the model as the necessary parameter. If you don't have the model, you can load it by

`model = gensim.models.Word2Vec.load('path/to/model')`

If you don't want to plot inside a notebook, set the `plot_in_notebook` parameter to `False`.

In [48]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go

def reduce_dimensions(model, plot_in_notebook = True):

    num_dimensions = 2        # final num dimensions (2D, 3D, etc)

    vectors = []         # positions in vector space
    labels = []         # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model[word])
        labels.append(word)


    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
    # reduce using t-SNE
    vectors = np.asarray(vectors)
    logging.info('starting tSNE dimensionality reduction. This may take some time.')
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [ v[0] for v in vectors ]
    y_vals = [ v[1] for v in vectors ]        
        
    # Create a trace
    trace = go.Scatter(
        x = x_vals,
        y = y_vals,
        mode = 'text',
        text = labels
        )
    
    data = [trace]
    
    logging.info('All done. Plotting.')
    
    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')

In [49]:
reduce_dimensions(model)

2017-06-22 11:06:03,601 : INFO : starting tSNE dimensionality reduction. This may take some time.
2017-06-22 11:06:11,212 : INFO : All done. Plotting.


## Conclusion

In this tutorial we learned how to train word2vec models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!

## Links


Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).